In [1]:
import json
import numpy as np
import pandas as pd
import pickle

In [30]:
def extract_data_from_json(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
        key_report_cas = data.get('key_report_cas', {})
        natrual_oils = data.get('natrual_oils', {})
        all_key_report_cas = data.get('all_key_report_cas', {})
        key_natrual_oils = data.get("key_recipe_code", {})

        return key_report_cas, natrual_oils, all_key_report_cas, key_natrual_oils

In [43]:

json_file = 'data/key_materials.json'  # Replace with your JSON file path
key_report_cas, natrual_oils, all_key_report_cas, key_natrual_oils = extract_data_from_json(json_file)
NUM_KEY_CAS = 110
NUM_ALL_KEY_CAS = 820
NUM_NATURAL_OILS = 235
NUM_KEY_OILS = 5

In [25]:

def extract_vector_from_report(key_report_cas, report):
    input_vector = np.zeros(NUM_KEY_CAS)
    report_data = report
    report_data = report_data[['CAS', '面积百分比']]
    report_data = report_data.groupby('CAS')['面积百分比'].sum().reset_index()
    for _, row in report_data.iterrows():
        cas = row['CAS']
        area = row['面积百分比']
        if cas in key_report_cas:
            input_vector[key_report_cas[cas]] = area   
    return input_vector

In [20]:

def extract_all_vector_from_report(key_report_cas, report):
    input_vector = np.zeros(NUM_ALL_KEY_CAS)
    report_data = report
    report_data = report_data[['CAS', '面积百分比']]
    report_data = report_data.groupby('CAS')['面积百分比'].sum().reset_index()
    for _, row in report_data.iterrows():
        cas = row['CAS']
        area = row['面积百分比']
        if cas in key_report_cas:
            input_vector[key_report_cas[cas]] = area   
    return input_vector

In [26]:
def extract_vector_from_recipe(natrual_oils, recipe):
    output_feature = np.zeros(NUM_NATURAL_OILS)
    for _, row in recipe.iterrows():
        code = str(row['元件品号'])
        percentage = row['组成用量']
        if code in natrual_oils:
            output_feature[natrual_oils[code]] = percentage
    return output_feature 

In [33]:
def extract_key_vector_from_recipe(natrual_oils, recipe):
    output_feature = np.zeros(NUM_KEY_OILS)
    for _, row in recipe.iterrows():
        code = str(row['元件品号'])
        percentage = row['组成用量']
        if code in natrual_oils:
            output_feature[natrual_oils[code]] = percentage
    return output_feature 

In [166]:
sample_report = 'test_sample/sample_report.xlsx'
sample_recipe = 'test_sample/sample_recipe.xlsx'

In [2]:
with open('data/database.pkl', 'rb') as f:
    loaded_data = pickle.load(f)
    analysisReportDf = loaded_data['analysisReportDf']
    formulaDf = loaded_data['formulaDf']
    analysisReportDf['面积百分比'] = analysisReportDf['面积百分比'].astype(float)
    formulaDf['组成用量'] = formulaDf['组成用量'].astype(float)

In [3]:
print(formulaDf.head())

     主件品号      主件品名                                  元件ID   元件品号  \
0  A00446  仿-粉香（水溶）  48DBFF66-04CB-4381-0672-154924518262  98086   
1  A00446  仿-粉香（水溶）  B3FCB979-1446-45BA-217B-1549245C316F  98061   
2  A00446  仿-粉香（水溶）  D305E665-DA05-4914-00AC-1549248F38EF  97099   
3  A00446  仿-粉香（水溶）  E7152084-CC7B-42A9-4AF3-154924A76F6E  97071   
4  A00446  仿-粉香（水溶）  F166E9CE-A38C-4783-C3ED-154924B32452  97055   

           元件品名        CAS号   组成用量   底数    Period          单位成本      成本金额  
0     异甲基紫罗兰酮70    127-51-5  0.220  100  2025-06   156.76243113  0.344877  
1        甲基紫罗兰酮    127-42-4  0.070  100  2025-06   223.47466395  0.156432  
2  羟基香草醛/羟醛（进口）    107-75-5  0.136  100  2025-06   334.96232957  0.455548  
3          波洁红醛  18127-01-0  0.080  100  2025-06   716.43315237  0.573146  
4          兔耳草醛    103-95-7  0.300  100  2025-06   314.34678257  0.943040  


In [4]:
print(analysisReportDf.head())

   Unnamed: 0      时间                              名称          CAS   匹配度  \
0         0.0   6.719                          丙二醇/PG  000057-55-6  91.0   
1         1.0   9.579  2-Propanol, 1-(2-propenyloxy)-  021460-36-6  81.0   
2         2.0  12.199         丁酸异丁酯/ISOBUTYL BUTYRATE  000539-90-2  79.0   
3         3.0  13.482                苯甲醛/BENZALDEHYDE  000100-52-7  63.0   
4         4.0  13.628               甲位蒎烯/ALPHA PINENE  000080-56-8  94.0   

   面积百分比     分析编号  
0  0.020  FC13521  
1  0.050  FC13521  
2  0.002  FC13521  
3  0.001  FC13521  
4  0.080  FC13521  


In [24]:
report = analysisReportDf[analysisReportDf['分析编号'] == 'FC13525'].reset_index(drop=True)
input_vector = extract_all_vector_from_report(all_key_report_cas, report)
print(input_vector)

[0.000e+00 0.000e+00 0.000e+00 2.600e-01 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 7.887e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 7.000e-02 0.000e+00 1.000e-03 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 5.000e-02 0.000e+00 2.000e-02
 0.000e+00 4.000e-03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 9.000e-02 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 7.400e-01 0.000e+00
 0.000e+00 4.000e-03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.970e+00 0.000e+00 0.000e+00
 0.000

In [171]:
recipe = formulaDf[formulaDf['主件品号'] == 'P13412'].reset_index(drop=True)
output_feature = extract_vector_from_recipe(natrual_oils, recipe)
print(output_feature)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [5]:
report_to_recipe = pd.read_excel('data/report_recipe.xlsx').reset_index(drop=True)

input_vectors = []
output_features = []
for _, row in report_to_recipe.iterrows():
    report_id = row['分析报告-分析编号']
    print(report_id)

    recipe_id = row['配方-调香师编号']

    
    report = analysisReportDf[analysisReportDf['分析编号'] == report_id].reset_index(drop=True)
    recipe = formulaDf[formulaDf['主件品号'] == recipe_id].reset_index(drop=True)
    
    input_vector = extract_vector_from_report(key_report_cas, report)
    output_feature = extract_key_vector_from_recipe(key_natrual_oils, recipe)

    if input_vector.sum() == 0 or output_feature.sum() == 0:
        continue

    input_vectors.append(input_vector)
    output_features.append(output_feature)

input_vectors = np.array(input_vectors)
output_features = np.array(output_features)

# Save the processed data
np.save('data/input.npy', input_vectors)
np.save('data/key_output.npy', output_features)

FC13525
FC13644
FC13527
FC13530
FC13531
FC13538
FC13544
FC13545
FC13557
FC13570
FC13572
FC13565
FC13566
FC13586
FC13584
FC13599
FC13596
FC13600
FC13629
FC13632
FC13633
FC13605
FC13638
FC13635
FC13652
FC13658
FC13656
FC13667
FC13675
FC13681
FC13689
FC13698
FC13697
FC13696
FC13699
FC13703
FC13711
FC13702
FC13714
FC13730
FC13729
FC13724
FC13717
FC13732
FC13734
FC13737
FC13739
FC13762
FC13765
FC13766
FC13772
FC13775
FC13778
FC13783
FC13814
FC13813
FC13812
FC13793
FC13817
FC13824
FC13823
FC13833
FC13835
FC14011
FC13840
FC13855
FC13863
FC13864
FC13875
FC13884
FC13879
FC13882
FC13886
FC13887
FC13893
FC13898
FC13899
FC13910
FC13927
FC13928
FC13917
FC13924
FC13932
FC13964
FC13939
FC13948
FC13952
FC13970
FC13991
FC14004
FC14005
FC14014
FC14021
FC14023
FC14030
FC14037
FC14046
FC14070
FC14071
FC14072
FC14050
FC14051
FC14052
FC14053
FC14084
FC14083
FC14081
FC14110
FC14116
FC14120
FC14139
FC13913
FC14175
FC14177
FC14179
FC14190
FC14234
FC14238
FC14259
FC14269
FC14263
FC14267
FC14307
FC14308
FC14318


In [48]:
print(input_vectors.shape)

(448, 110)


In [49]:
print(output_features.shape)


(448, 5)


In [126]:
import torch
from model import SimpleNN
from sklearn.preprocessing import StandardScaler

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [155]:
# 1. Define model structure again
model = SimpleNN()
model.load_state_dict(torch.load("compound_to_oil_model.pth"))
model.eval()
model.to(device)

# 2. Inference
x_new = input_vectors[5]
x_new = x_new.reshape(1, -1)  # Reshape to match model input
x_new = np.log1p(x_new)

y_new = output_features[5]
y_new = y_new.reshape(1, -1)  # Reshape to match model output
y_new = np.log1p(y_new)

x_tensor = torch.tensor(x_new, dtype=torch.float32).to(device)

with torch.no_grad():
    y_pred = model(x_tensor)
    y_pred_np = y_pred.cpu().numpy()

# 3. Reverse output transformation
y_pred_original = np.expm1(y_pred_np)

In [159]:
y_pred_np = y_pred_np.reshape(-1)
y_pred_np[y_pred_np < 1e-5] = 0
print(y_pred_np)
print(y_new)

[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 9.7746837e-05
 0.0000000e+00 1.7617580e-04 0.0000000e+00 2.6866165e-04 6.0051528e-04
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 1.7172842e-03 0.0000000e+00 1.6709609e-02
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.000

In [160]:
y = output_features[0]
print(np.mean((y - y_pred_original) ** 2))

0.02153033398463458


In [4]:
all_key_cas = pd.read_excel('data/PEP天然油识别(1).xlsx', sheet_name='主要CAS总表8.15').reset_index(drop=True)

In [10]:
list_all_key_cas = list(all_key_cas['CAS'])
print(len(list_all_key_cas))
dict_all_key_cas = {}
count = 0
for cas in list_all_key_cas:
    dict_all_key_cas[cas] = count
    count += 1

820


In [14]:
with open('data/key_materials.json', 'r') as file:
    data = json.load(file)

print(type(data))

<class 'dict'>


In [15]:
data['all_key_report_cas'] = dict_all_key_cas

In [16]:
with open("data.json", "w") as f:
    json.dump(data, f, indent=4)